# Система детекции людей на тепловизионных изображениях

## 📊 Сравнительный анализ тепловых датасетов

| **Параметр** | **FLIR ADAS (thermal)** | **KAIST Multispectral** | **COCO (RGB fallback)** |
|--------------|-------------------------|-------------------------|-------------------------|
| **Количество изображений** | ~10 000 тепловых кадров | ~95 000 (синхронизированные пары) | ~330 000 |
| **Классы** | Люди, транспорт, фон | Люди, транспорт, RGB+Thermal | 80 классов (включая людей) |
| **Тепловые метки** | Температурная карта + bbox | Термальные и RGB bbox | Нет термальной компоненты |
| **Подходящий сценарий** | Периметровая охрана, пожарная безопасность | Мультиспектральный мониторинг пешеходов | Общая детекция объектов в RGB-сценах |
| **Авторитетность** | FLIR OEM (Lin et al., 2016) | KAIST Research (Hwang et al., 2015) | Стандартизованный COCO (Lin et al., 2014) |

**Выбор:** для нашей системы мы используем **FLIR ADAS** как основной набор данных, поскольку он содержит реальную термальную информацию и описывает сценарии, близкие к требованиям лабораторной работы. COCO сохраняем как контрольный RGB-базис для сравнений.

## 1. Установка и импорт необходимых библиотек

Рабочая база — термальный FLIR ADAS (либо аннотированный аналог), поэтому устанавливаем инструменты для обработки изображений, преобразования аннотаций и визуализации распознанных людей.


In [ ]:

import kagglehub
deepnewbie_flir_thermal_images_dataset_path = kagglehub.dataset_download('deepnewbie/flir-thermal-images-dataset')

print('Data source import complete.')
print(deepnewbie_flir_thermal_images_dataset_path)


In [ ]:
import json
import os
from collections import defaultdict

# Датасет настройка путей
json_path = "/root/.cache/kagglehub/datasets/deepnewbie/flir-thermal-images-dataset/versions/1/FLIR_ADAS_1_3/train/thermal_annotations.json"
images_dir = "/root/.cache/kagglehub/datasets/deepnewbie/flir-thermal-images-dataset/versions/1/FLIR_ADAS_1_3/train/Annotated_thermal_8_bit"
output_dir = "/kaggle/working/labels_yolo"
os.makedirs(output_dir, exist_ok=True)

# JSON згружаем
with open(json_path, "r") as f:
    data = json.load(f)

# image_id -> filename
image_id_to_info = {}
for img in data['images']:
    image_id_to_info[img['id']] = {
        'file_name': img['file_name'],
        'width': img['width'],
        'height': img['height']
    }

# перевода ID категории в название
category_id_to_name = {cat['id']: cat['name'] for cat in data['categories']}

# Загрузка классов выбранных людей и машин
class_map = {
    'person': 0,
    'car': 1
}

# Annotations -> YOLO format
labels = defaultdict(list)
for ann in data['annotations']:
    img_id = ann['image_id']
    cat_id = ann['category_id']
    bbox = ann['bbox']  # [x, y, width, height]


##  Подключения Google Диска к Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

labels_path = "/kaggle/working/labels_yolo"

yolo_files = []
for root, _, files in os.walk(labels_path):
    for file in files:
        if file.endswith(".txt"):
            yolo_files.append(os.path.join(root, file))


yolo_files.sort()

for file_path in yolo_files[:2]:
    print(f"\n---- {file_path} ----")
    with open(file_path, "r") as f:
        print(f.read())


Определение путей к меткам, поиск файлов с разметкой, сортировка, просмотр содержимого первых файлов

In [ ]:
import json
import os
import shutil
from collections import defaultdict

# Структура папок YOLO внутреняя
dataset_root = "/kaggle/working/datasets"
label_train_dir = os.path.join(dataset_root, "labels/train")
label_val_dir = os.path.join(dataset_root, "labels/val")
image_train_dir = os.path.join(dataset_root, "images/train")
image_val_dir = os.path.join(dataset_root, "images/val")

for path in [label_train_dir, label_val_dir, image_train_dir, image_val_dir]:
    os.makedirs(path, exist_ok=True)

# Пути к JSON и изображениям
json_paths = {
    "train": "/root/.cache/kagglehub/datasets/deepnewbie/flir-thermal-images-dataset/versions/1/FLIR_ADAS_1_3/train/thermal_annotations.json",
    "val": "/root/.cache/kagglehub/datasets/deepnewbie/flir-thermal-images-dataset/versions/1/FLIR_ADAS_1_3/val/thermal_annotations.json"
}
image_dirs = {
    "train": "/root/.cache/kagglehub/datasets/deepnewbie/flir-thermal-images-dataset/versions/1/FLIR_ADAS_1_3/train",
    "val": "/root/.cache/kagglehub/datasets/deepnewbie/flir-thermal-images-dataset/versions/1/FLIR_ADAS_1_3/val"
}

# category_id -> name
class_map = {'person': 0, 'car': 1} #0пределение классов

for split in ["train", "val"]:
    print(f"\n📂 Обрабатывается {split.upper()} dataset...")

    # Загрузка json
    with open(json_paths[split], "r") as f:
        data = json.load(f)

    # ID изображения -> информация об изображении
    image_id_to_info = {}
    for img in data['images']:
        image_id_to_info[img['id']] = {
            'file_name': img['file_name'],
            'width': img['width'],
            'height': img['height']
        }

    # ID категории -> название категории
    category_id_to_name = {cat['id']: cat['name'] for cat in data['categories']}

    # ID изображения -> строки меток
    labels = defaultdict(list)
    for ann in data['annotations']:
        img_id = ann['image_id']
        cat_id = ann['category_id']
        bbox = ann['bbox']
        cat_name = category_id_to_name[cat_id]
        # Пропустить классы, которых нет в class_map
        if cat_name not in class_map:
            continue

        x, y, w, h = bbox #Бокс
        info = image_id_to_info[img_id]
        img_w, img_h = info['width'], info['height']

        # Фильтрация маленьких объектов
        if w < 5 or h < 5 or w / img_w < 0.01 or h / img_h < 0.01: #удаляет объекты меньше 5 пикселей и объекты меньше 1% от ширины изображения
            continue
        # Фильтрация вытянутых объектов
        if w / h > 10 or h / w > 10: # удаляет слишком вытянутые по горизонтали объекты и слишком вытянутые по вертикали объекты
            continue
        # Конвертация в формат YOLO (нормализованные координаты)
        x_center = (x + w/2) / img_w
        y_center = (y + h/2) / img_h
        w_norm = w / img_w
        h_norm = h / img_h

        labels[img_id].append(
            f"{class_map[cat_name]} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}"
        )

    # Сохранение меток и копирование изображений
    for img_id, info in image_id_to_info.items():
        file_name = os.path.basename(info['file_name'])
        txt_file_name = os.path.splitext(file_name)[0] + ".txt"

        if split == "train":
            txt_path = os.path.join(label_train_dir, txt_file_name)
            img_dst_path = os.path.join(image_train_dir, file_name)
        else:
            txt_path = os.path.join(label_val_dir, txt_file_name)
            img_dst_path = os.path.join(image_val_dir, file_name)

        # Запись меток
        with open(txt_path, "w") as f:
            if img_id in labels:
                f.write("\n".join(labels[img_id]) + "\n")

        # Копирование изображений
        orig_img_path = os.path.join(image_dirs[split], info['file_name'])
        if os.path.exists(orig_img_path):
            shutil.copy(orig_img_path, img_dst_path)
        else:
            print(f"⚠ Изображение не найдено:: {orig_img_path}")

print("\n JSON -> YOLO датасет полностью подготовлен.")


In [ ]:
import os
import shutil

# Пути к папкам
image_val_dir = "/kaggle/working/datasets/images/val"
label_val_dir = "/kaggle/working/datasets/labels/val"
image_test_dir = "/kaggle/working/datasets/images/test"
label_test_dir = "/kaggle/working/datasets/labels/test"

# Создание тестовых папок
os.makedirs(image_test_dir, exist_ok=True)
os.makedirs(label_test_dir, exist_ok=True)

# Получить все изображения из val папки и отсортировать
images = sorted([f for f in os.listdir(image_val_dir) if f.lower().endswith(('.jpg','.jpeg','.png'))])

# Разделение набора
split_index = 500 # первые изображений остаются в validation, остальные тест
val_images = images[:split_index]
test_images = images[split_index:]

# Перемещение выбранных файлов в test
for img_file in test_images:
    # Переместить изображение
    shutil.move(
        os.path.join(image_val_dir, img_file),
        os.path.join(image_test_dir, img_file)
    )
    # Переместить метку
    txt_file = os.path.splitext(img_file)[0] + ".txt"
    shutil.move(
        os.path.join(label_val_dir, txt_file),
        os.path.join(label_test_dir, txt_file)
    )

print(f" В папке val было всего: {len(images)} изображений.")
print(f"Теперь val: {len(val_images)} изображений, test: {len(test_images)} изображений.")

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os

# Пути к папкам
dataset_root = "/kaggle/working/datasets"
image_train_dir = os.path.join(dataset_root, "images/train")
image_val_dir = os.path.join(dataset_root, "images/val")
image_test_dir = os.path.join(dataset_root, "images/test")

# Найти первое изображение в каждой папке
def get_first_image(directory):
    images = sorted([f for f in os.listdir(directory) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    if images:
        return os.path.join(directory, images[0])
    else:
        return None
#Получение первых изображений
train_image = get_first_image(image_train_dir)
val_image = get_first_image(image_val_dir)
test_image = get_first_image(image_test_dir)

# графика для визуализации
plt.figure(figsize=(15,5))

# Train изображение
plt.subplot(1, 3, 1)
if train_image:
    img = cv2.imread(train_image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img) ## Отобразить изображение
    plt.title("First Train Image")
    plt.axis("off")
else:
    ## Если изображения нет, показать текст
    plt.text(0.5, 0.5, "No train image", ha='center', va='center', fontsize=12)
    plt.axis("off")

# Val изображение
plt.subplot(1, 3, 2)
if val_image:
    img = cv2.imread(val_image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.title("First Val Image")
    plt.axis("off")
else:
    plt.text(0.5, 0.5, "No val image", ha='center', va='center', fontsize=12)
    plt.axis("off")

# Test
plt.subplot(1, 3, 3)
if test_image:
    img = cv2.imread(test_image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.title("First Test Image")
    plt.axis("off")
else:
    plt.text(0.5, 0.5, "No test image", ha='center', va='center', fontsize=12)
    plt.axis("off")

plt.tight_layout()
plt.show() #Показ результата

## Этот код создает YAML-файл конфигурации для YOLO - необходимый файл для обучения и валидации модели YOLO.

In [ ]:
import os

yaml_content = """
path: /kaggle/working/datasets
train: images/train
val: images/val

nc: 2
names: ['person', 'car']
"""

file_path = "/kaggle/working/thermal.yaml"

with open(file_path, "w") as f:
    f.write(yaml_content.strip())

# Получаем абсолютный путь
abs_path = os.path.abspath(file_path)
print(f"Файл thermal.yaml создан")
print(f" Полный путь: {abs_path}")


## Проверка структуры датасета YOLO и вывод информации о наличии файлов в каждой папке.

In [ ]:
import os

def check_dir_structure(base_dir):
    folders = [
        "images/train", "images/val",
        "labels/train", "labels/val"
    ]
    for folder in folders:
        path = os.path.join(base_dir, folder)
        if not os.path.exists(path):
            print(f" {folder} Не существует.")
        else:
            files = os.listdir(path)
            image_files = [f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff'))]
            label_files = [f for f in files if f.lower().endswith('.txt')]

            print(f"В {folder}:")
            print(f"    общее количество файлов: {len(files)}")
            if "images" in folder:
                print(f"    количество изображений: {len(image_files)}")
            else:
                print(f"    количество текстовых файлов : {len(label_files)}")
            if len(files) == 0:
                print("    ⚠ папка пустая!")

# Вызов функции проверки структуры по пути
check_dir_structure("/kaggle/working/datasets")


# Устанавливает библиотеку Ultralytics YOLO - фреймворк для работы с моделями YOLO

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11m.pt")  # Загрузка модели

# Оптимизированные параметры для тепловизионных данных
model.train(
    data="/kaggle/working/thermal.yaml",
    epochs=5,
    imgsz=64, # Масштаб изображений для лучшего определения лучше 320-640 для средней и 640-1280 для большой
    batch=16,
    patience=10,
    degrees=5,      # Без вращения
    scale=0.2,      # %20 Масштабирование
    shear=1,        # Без сдвига
    perspective=0,  # Без перспективных искажений
    flipud=0,       # Без вертикального отражения
    hsv_h=0, hsv_s=0, hsv_v=0, # Только яркость
    mosaic=0.5,     # Аугментация
    mixup=0.05,     #
    lr0=0.001,      #
    lrf=0.1,
    warmup_epochs=3

)
#device=0

In [ ]:
import os
# Путь к данным
img_dir = "/root/.cache/kagglehub/datasets/deepnewbie/flir-thermal-images-dataset/versions/1/FLIR_ADAS_1_3/val/thermal_8_bit"

# Сбор всех JPEG-изображений
all_images = [os.path.join(img_dir, fname) for fname in sorted(os.listdir(img_dir)) if fname.endswith(".jpeg")]

# Выбор изображений для теста
test_images = all_images[500:]

print(f"Всего {len(all_images)} изображений.")
print(f"Первые500 пропущены, для теста будет {len(test_images)} изображений.")

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
from collections import Counter

# Загрузка модели
model = YOLO("/content/yolo11m.pt")
class_map = model.names  # ör: {0: 'person', 1: 'car'}

# Пути к данным
img_dir_test = "/kaggle/working/datasets/images/test"
label_dir_test = "/kaggle/working/datasets/labels/test"
img_dir_annotated = "/root/.cache/kagglehub/datasets/deepnewbie/flir-thermal-images-dataset/versions/1/FLIR_ADAS_1_3/val/Annotated_thermal_8_bit"

#  Выбор тестовых изображений (первые 20 для примера)
test_images = sorted([f for f in os.listdir(img_dir_annotated) if f.endswith(('.jpeg','.jpg','.png'))])[-20:]
print(f"Тестируем на {len(test_images)} изображенияз ")

#  Классы
gt_class_map = {0: 'person', 1: 'car',2: 'bicycle', 3:'dog'}

for file_name in test_images[:100]:
    img_path_annotated = os.path.join(img_dir_annotated, file_name)
    img_path_test = os.path.join(img_dir_test, file_name)
    label_path_test = os.path.join(label_dir_test, os.path.splitext(file_name)[0] + ".txt")

    # Чтение изображений
    img_annotated = cv2.imread(img_path_annotated)
    img_annotated = cv2.cvtColor(img_annotated, cv2.COLOR_BGR2RGB)
    img_test = cv2.imread(img_path_test)
    img_test = cv2.cvtColor(img_test, cv2.COLOR_BGR2RGB)

    # ================================
 # Анализ реальной разметки
    gt_classes = []
    if os.path.exists(label_path_test):
        with open(label_path_test, "r") as f:
            lines = f.readlines()
        for line in lines:
            cls_id, _, _, _, _ = map(float, line.strip().split())
            cls_name = gt_class_map.get(int(cls_id), f"id:{int(cls_id)}")
            gt_classes.append(cls_name)
        gt_count = Counter(gt_classes)
        print(f"\n {file_name} | Ground Truth Counts: {dict(gt_count)}")
    else:
        print(f"\n {file_name} | No ground truth labels found!")

    # ================================
    # Создание графика
    fig, axs = plt.subplots(1, 2, figsize=(14, 7))

    # Left: Аннотированное изображение
    axs[0].imshow(img_annotated)
    axs[0].set_title(f"Annotated GT: {file_name}")
    axs[0].axis("off")

    # Right : Предсказания модели
    axs[1].imshow(img_test)
    axs[1].set_title(f"Prediction: {file_name}")
    axs[1].axis("off")

# Получение предсказаний от модели
    results = model.predict(img_path_test, save=False, imgsz=640, conf=0.25)
    preds = results[0].boxes
    for box, cls_id in zip(preds.xywh, preds.cls):
        xc, yc, bw, bh = box.tolist()
        x1 = (xc - bw/2)
        y1 = (yc - bh/2)
        rect = patches.Rectangle((x1, y1), bw, bh, linewidth=1.5, edgecolor='orange', facecolor='none')
        axs[1].add_patch(rect)
        axs[1].text(x1, y1, class_map.get(int(cls_id), str(int(cls_id))), color='orange', fontsize=8, weight='bold')

    plt.tight_layout()
    plt.show()

In [ ]:
from ultralytics import YOLO
import cv2
import os
from collections import Counter

# Загрузка модели
model = YOLO("/content/yolo11m.pt")
class_map = model.names

# Определение путей
img_dir_test = "/kaggle/working/datasets/images/test"
label_dir_test = "/kaggle/working/datasets/labels/test"
img_dir_annotated = "/root/.cache/kagglehub/datasets/deepnewbie/flir-thermal-images-dataset/versions/1/FLIR_ADAS_1_3/val/Annotated_thermal_8_bit"

 # Выбор тестовых изображений
test_images = sorted([f for f in os.listdir(img_dir_annotated) if f.endswith(('.jpeg','.jpg','.png'))])[-100:]
print(f"Выбрано последних изображений для теста. Всего: {len(test_images)}")

gt_class_map = {0: 'person', 1: 'car'}
# Список для хранения accuracy по каждому изображению
frame_accuracies = []
# Обработка каждого тестового изображения
for file_name in test_images:
    img_path_test = os.path.join(img_dir_test, file_name)
    label_path_test = os.path.join(label_dir_test, os.path.splitext(file_name)[0] + ".txt")

    img_test = cv2.imread(img_path_test)
    results = model.predict(img_test, save=False, imgsz=640, conf=0.25)
    preds = results[0].boxes
    pred_classes = [class_map.get(int(cls_id), str(int(cls_id))) for cls_id in preds.cls]
    pred_counter = Counter(pred_classes)

    gt_classes = []
    if os.path.exists(label_path_test):
        with open(label_path_test, "r") as f:
            for line in f:
                cls_id, *_ = map(float, line.strip().split())
                gt_classes.append(gt_class_map.get(int(cls_id), f"id:{int(cls_id)}"))
    gt_counter = Counter(gt_classes)

    # # Расчет accuracy для текущего изображения (кадра)
    acc_list = []
    for cls in gt_counter:
        gt_count = gt_counter[cls]
        pred_count = pred_counter.get(cls, 0)
        acc_list.append(min(pred_count, gt_count) / gt_count)

    # Усреднение accuracy по всем классам в изображении
    if acc_list:
        frame_acc = sum(acc_list) / len(acc_list)
    else:
        frame_acc = 1.0  # # Если нет ground truth, считаем accuracy = 100% (спорное решение)

    frame_accuracies.append(frame_acc)

# Расчет общей средней accuracy по всем изображениям
overall_acc = sum(frame_accuracies) / len(frame_accuracies) if frame_accuracies else 0.0
print(f"\n🎯  Средняя accuracy по всем изображениям: {overall_acc*100:.2f}%")


In [ ]:
from ultralytics import YOLO
import cv2
import os

# Загрузка модели
model = YOLO("/content/yolo11m.pt")
class_map = model.names

# Определение папки с кадрами видео
frame_folder = "/root/.cache/kagglehub/datasets/deepnewbie/flir-thermal-images-dataset/versions/1/FLIR_ADAS_1_3/video/thermal_8_bit"
frames = sorted([f for f in os.listdir(frame_folder) if f.endswith('.jpeg')])

for f in frames:
    img_path = os.path.join(frame_folder, f)
    frame = cv2.imread(img_path)

    # Получение предсказаний модели
    results = model.predict(frame, imgsz=640, conf=0.25, verbose=False)
    preds = results[0].boxes


    for box, cls_id in zip(preds.xywh, preds.cls):
        xc, yc, bw, bh = box.tolist()
        x1 = int(xc - bw/2)
        y1 = int(yc - bh/2)
        x2 = int(xc + bw/2)
        y2 = int(yc + bh/2)

        label = class_map.get(int(cls_id), str(int(cls_id)))
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,140,255), 2)
        cv2.putText(frame, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (0,140,255), 2, cv2.LINE_AA)

    # Показать кадр с детекцией
    cv2.imshow("Thermal Detection", frame)

    # Ожидание нажатия клавиши
    if cv2.waitKey(60) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

In [ ]:
from ultralytics import YOLO
import cv2
import os


model = YOLO("/content/yolo11m.pt")


class_map = model.names


frame_folder = "/root/.cache/kagglehub/datasets/deepnewbie/flir-thermal-images-dataset/versions/1/FLIR_ADAS_1_3/video/thermal_8_bit"
frames = sorted([f for f in os.listdir(frame_folder) if f.endswith('.jpeg')])

for f in frames:
    img_path = os.path.join(frame_folder, f)
    frame = cv2.imread(img_path)

    # YOLO tahmin
    results = model.predict(frame, imgsz=640, conf=0.25, verbose=False)
    preds = results[0].boxes

    # Kutuları çiz
    for box, cls_id in zip(preds.xywh, preds.cls):
        xc, yc, bw, bh = box.tolist()
        x1, y1 = int(xc - bw/2), int(yc - bh/2)
        x2, y2 = int(xc + bw/2), int(yc + bh/2)
        label = class_map.get(int(cls_id), str(int(cls_id)))
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255,140,0), 2)
        cv2.putText(frame, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (255,140,0), 2, cv2.LINE_AA)

    # Göster
    cv2.imshow("Termal YOLO Detection", frame)
    if cv2.waitKey(60) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()